In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn import tree
from sklearn.tree import plot_tree, export_text

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

In [ ]:
tweets = pd.read_csv("data/tweets.csv")
words = pd.read_csv("data/words_freq.csv")

In [5]:
max(words["occurrence"])

0.0024305603723041

In [6]:
tweets["score"] = tweets["tweet_text"].str[11]
tweets["score"] = pd.to_numeric(tweets['score'], errors='coerce')
tweets.rename(columns={"wordle_id": "day"}, inplace=True)
tweets

,day,tweet_id,tweet_date,tweet_username,tweet_text,score
0,210,1482553374591660037,2022-01-16 03:20:43+00:00,bpszebes,Wordle 210 4/6\n\n⬛⬛🟨🟨⬛\n🟩⬛⬛⬛⬛\n🟩🟩🟨⬛⬛\n🟩🟩🟩🟩🟩,4
1,210,1482553387937898499,2022-01-16 03:20:46+00:00,cruisecoup,Wordle 210 4/6\n\n⬜⬜⬜⬜⬜\n🟩🟩🟨🟨⬜\n🟩🟩⬜🟩🟨\n🟩🟩🟩🟩🟩,4
2,210,1482553422276698113,2022-01-16 03:20:55+00:00,DestroVega,Wordle 210 4/6\n\n⬜⬜⬜🟨⬜\n⬜🟩⬜🟨⬜\n⬜🟩⬜🟨⬜\n🟩🟩🟩🟩🟩,4
3,210,1482553436910628866,2022-01-16 03:20:58+00:00,brenmardash,Wordle 210 3/6\n\n⬜⬜🟨⬜⬜\n🟨🟨⬜⬜⬜\n🟩🟩🟩🟩🟩,3
4,210,1482553445726908420,2022-01-16 03:21:00+00:00,KatieHowse2,Wordle 210 3/6\n\n⬛⬛🟨⬛⬛\n🟩🟩🟩⬛⬛\n🟩🟩🟩🟩🟩,3
...,...,...,...,...,...,...
1178449,519,1594492773662871553,2022-11-21 00:47:56+00:00,MaureenLamont,Wordle 519 2/6\n\n⬜🟩🟨🟨⬜\n🟩🟩🟩🟩🟩,2
1178450,519,1594492932115124226,2022-11-21 00:48:33+00:00,Meeshbeer,Wordle 519 3/6\n\n⬜🟨🟩🟨⬜\n⬜🟩🟩🟩🟩\n🟩🟩🟩🟩🟩,3
1178451,519,1594493005192658944,2022-11-21 00:48:51+00:00,BookChickie,Wordle 519 3/6\n\n⬛🟩🟩⬛🟩\n🟩🟩🟩⬛🟩\n🟩🟩🟩🟩🟩,3
1178452,519,1594493051824754689,2022-11-21 00:49:02+00:00,aceynay,Wordle 519 3/6\n\n🟨🟩⬜⬜🟩\n🟩🟩🟩⬜🟩\n🟩🟩🟩🟩🟩\n\nI pla...,3


In [7]:
tweets.isnull().sum()

day               0
tweet_id          0
tweet_date        0
tweet_username    0
tweet_text        0
score             0
dtype: int64

In [8]:
words.dropna(inplace=True)
words["day"] = pd.to_numeric(words['day'], errors='coerce')

In [9]:
words.isnull().sum()

word          0
occurrence    0
day           0
dtype: int64

In [10]:
df = pd.merge(words, tweets, on='day')
df.drop(columns=['tweet_id'], inplace=True)
df

,word,occurrence,day,tweet_date,tweet_username,tweet_text,score
0,admit,1.820914e-05,485.0,2022-10-17 16:05:17+00:00,nickdeephoto,Wordle 485 2/6\n\n🟩🟩⬛🟨🟨\n🟩🟩🟩🟩🟩,2
1,admit,1.820914e-05,485.0,2022-10-17 16:05:18+00:00,Green_Mt_Girl,Wordle 485 4/6\n\n⬜⬜⬜🟩⬜\n⬜🟨🟨🟩🟨\n🟨⬜⬜🟩🟩\n🟩🟩🟩🟩🟩,4
2,admit,1.820914e-05,485.0,2022-10-17 16:05:20+00:00,Manab_Deka911,Wordle 485 2/6\n\n🟩⬛⬛🟩🟩\n🟩🟩🟩🟩🟩,2
3,admit,1.820914e-05,485.0,2022-10-17 16:05:23+00:00,Aquagenerian,Wordle 485 3/6\n\n🟨⬛⬛🟨🟨\n🟩⬛⬛🟨⬛\n🟩🟩🟩🟩🟩,3
4,admit,1.820914e-05,485.0,2022-10-17 16:05:54+00:00,grcflwlkr,Wordle 485 4/6\n\n🟩⬜🟩⬜⬜\n⬜🟨⬜⬜🟨\n🟩⬜🟨🟨🟨\n🟩🟩🟩🟩🟩\n...,4
...,...,...,...,...,...,...,...
1178449,zesty,7.402564e-08,319.0,2022-05-05 02:43:24+00:00,KevinAWortman,Wordle 319 2/6\n\n🟨⬜🟨🟨⬜\n🟩🟩🟩🟩🟩,2
1178450,zesty,7.402564e-08,319.0,2022-05-05 02:43:27+00:00,samsjag,Wordle 319 2/6\n\n🟩⬛🟩🟨⬛\n🟩🟩🟩🟩🟩,2
1178451,zesty,7.402564e-08,319.0,2022-05-05 02:43:30+00:00,msghiorso,Wordle 319 3/6\n\n⬜🟨🟨⬜🟨\n🟨⬜⬜🟩🟩\n🟩🟩🟩🟩🟩,3
1178452,zesty,7.402564e-08,319.0,2022-05-05 02:43:32+00:00,katgarber,Wordle 319 5/6\n\n⬜🟨⬜🟨⬜\n🟩⬜🟨🟨⬜\n🟩⬜🟩🟨⬜\n🟩⬜🟩🟩🟩\n...,5


In [11]:
"""
For the word in each row:

    1. Put the word in lower case
    2. Extract each letter in the word and make it it's own column
    3. Convert to ASCII number using ord() function
    4. subtract 96 to simplify char to number representation (a = 1, b = 2, c = 3, ...)

"""

df["letter_1"] = df["word"].str.lower().str[0].apply(ord) - 96
df["letter_2"] = df["word"].str.lower().str[1].apply(ord) - 96
df["letter_3"] = df["word"].str.lower().str[2].apply(ord) - 96
df["letter_4"] = df["word"].str.lower().str[3].apply(ord) - 96
df["letter_5"] = df["word"].str.lower().str[4].apply(ord) - 96
# df.drop(columns=["word"], inplace=True)
df

,word,occurrence,day,tweet_date,tweet_username,tweet_text,score,letter_1,letter_2,letter_3,letter_4,letter_5
0,admit,1.820914e-05,485.0,2022-10-17 16:05:17+00:00,nickdeephoto,Wordle 485 2/6\n\n🟩🟩⬛🟨🟨\n🟩🟩🟩🟩🟩,2,1,4,13,9,20
1,admit,1.820914e-05,485.0,2022-10-17 16:05:18+00:00,Green_Mt_Girl,Wordle 485 4/6\n\n⬜⬜⬜🟩⬜\n⬜🟨🟨🟩🟨\n🟨⬜⬜🟩🟩\n🟩🟩🟩🟩🟩,4,1,4,13,9,20
2,admit,1.820914e-05,485.0,2022-10-17 16:05:20+00:00,Manab_Deka911,Wordle 485 2/6\n\n🟩⬛⬛🟩🟩\n🟩🟩🟩🟩🟩,2,1,4,13,9,20
3,admit,1.820914e-05,485.0,2022-10-17 16:05:23+00:00,Aquagenerian,Wordle 485 3/6\n\n🟨⬛⬛🟨🟨\n🟩⬛⬛🟨⬛\n🟩🟩🟩🟩🟩,3,1,4,13,9,20
4,admit,1.820914e-05,485.0,2022-10-17 16:05:54+00:00,grcflwlkr,Wordle 485 4/6\n\n🟩⬜🟩⬜⬜\n⬜🟨⬜⬜🟨\n🟩⬜🟨🟨🟨\n🟩🟩🟩🟩🟩\n...,4,1,4,13,9,20
...,...,...,...,...,...,...,...,...,...,...,...,...
1178449,zesty,7.402564e-08,319.0,2022-05-05 02:43:24+00:00,KevinAWortman,Wordle 319 2/6\n\n🟨⬜🟨🟨⬜\n🟩🟩🟩🟩🟩,2,26,5,19,20,25
1178450,zesty,7.402564e-08,319.0,2022-05-05 02:43:27+00:00,samsjag,Wordle 319 2/6\n\n🟩⬛🟩🟨⬛\n🟩🟩🟩🟩🟩,2,26,5,19,20,25
1178451,zesty,7.402564e-08,319.0,2022-05-05 02:43:30+00:00,msghiorso,Wordle 319 3/6\n\n⬜🟨🟨⬜🟨\n🟨⬜⬜🟩🟩\n🟩🟩🟩🟩🟩,3,26,5,19,20,25
1178452,zesty,7.402564e-08,319.0,2022-05-05 02:43:32+00:00,katgarber,Wordle 319 5/6\n\n⬜🟨⬜🟨⬜\n🟩⬜🟨🟨⬜\n🟩⬜🟩🟨⬜\n🟩⬜🟩🟩🟩\n...,5,26,5,19,20,25


In [12]:
def inspect_dataframe(input_df):
    print("Data overview:")
    print(input_df.describe(), '\n')

    print("Null counts:")
    print(df.isnull().sum(), '\n')

    print("Duplicate row count:", df.duplicated().sum())

inspect_dataframe(df)

Data overview:
         occurrence           day         score      letter_1      letter_2  \
count  1.178454e+06  1.178454e+06  1.178454e+06  1.178454e+06  1.178454e+06   
mean   3.371276e-05  3.475489e+02  4.126197e+00  1.185610e+01  1.173020e+01   
std    1.325442e-04  9.180824e+01  1.077497e+00  7.345991e+00  6.420807e+00   
min    1.427564e-08  2.100000e+02  2.000000e+00  1.000000e+00  1.000000e+00   
25%    4.708097e-07  2.600000e+02  3.000000e+00  5.000000e+00  8.000000e+00   
50%    1.811237e-06  3.400000e+02  4.000000e+00  1.300000e+01  1.200000e+01   
75%    8.992359e-06  4.260000e+02  5.000000e+00  1.900000e+01  1.600000e+01   
max    1.333864e-03  5.190000e+02  6.000000e+00  2.600000e+01  2.500000e+01   

           letter_3      letter_4      letter_5  
count  1.178454e+06  1.178454e+06  1.178454e+06  
mean   1.199378e+01  1.181678e+01  1.309581e+01  
std    7.009741e+00  6.749040e+00  7.508834e+00  
min    1.000000e+00  1.000000e+00  1.000000e+00  
25%    5.000000e+00  5.

In [75]:
independent_variables = ['letter_1', 'letter_2', 'letter_3', 'letter_4', 'letter_5', 'occurrence']
dependent_variable = 'score'

X = df[independent_variables]
y = df[dependent_variable]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=45)
print('Shape of our X_train data:', X_train.shape, '\nShape of our y_test data:', y_test.shape)

Shape of our X_train data: (942763, 6) 
Shape of our y_test data: (235691,)


In [83]:
model = RandomForestRegressor(oob_score=True)

model.fit(X_train, y_train)

oob_score = model.oob_score_
print(f'Out-of-bag score: {oob_score}')

# Model evaluation
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean absolute error: {mae}')

mse = mean_squared_error(y_test, y_pred)
print(f'Mean squared error: {mse}')

rmse = root_mean_squared_error(y_test, y_pred)
print(f'Root mean squared error: {mse}')

r2 = r2_score(y_test, y_pred)
print(f'R-squared: {r2}')

Out-of-bag score: 0.0752127525212829
Mean absolute error: 0.8392852707758599
Mean squared error: 1.0722424983623786
Root mean squared error: 1.0722424983623786
R-squared: 0.07644356144155506


In [15]:
params = {
    'max_depth': [5, 10, 15], 
    'max_leaf_nodes': [5, 10, 15],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2, 3]
}

In [ ]:
# grid_search_cv =  GridSearchCV(RandomForestRegressor(), param_grid=params, scoring="neg_mean_squared_error")
# grid_search_cv.fit(X_train, y_train)

In [31]:
filename = 'wordle_prediction.pkl'
pickle.dump(model, open(filename, 'wb'))

In [32]:
model = pickle.load(open(filename, 'rb'))

In [33]:
def predict_score (word, occurrence):
    
    df = pd.DataFrame()
    df["word"] = [ word ]
    df["letter_1"] = df["word"].str.lower().str[0].apply(ord) - 96
    df["letter_2"] = df["word"].str.lower().str[1].apply(ord) - 96
    df["letter_3"] = df["word"].str.lower().str[2].apply(ord) - 96
    df["letter_4"] = df["word"].str.lower().str[3].apply(ord) - 96
    df["letter_5"] = df["word"].str.lower().str[4].apply(ord) - 96

    df["occurrence"] = [ occurrence ]
    df.drop(columns=["word"], inplace=True)

    return model.predict(df)
    

In [93]:
averages = df.groupby("word", as_index=False)['score'].mean()
averages

,word,score
0,admit,3.697055
1,agape,4.025042
2,agora,4.079151
3,ahead,3.814139
4,album,4.235260
...,...,...
301,woven,3.843087
302,wrung,4.246919
303,yield,4.059162
304,youth,4.138001


In [112]:
def get_scores(word, occurrence):
    
    prediction = predict_score (word, occurrence) [0]

    # If word isn't found in tweet data, None is returned for the average score
    average = None
    if word in averages["word"].values:
        average = averages[averages["word"] == word]["score"].item()

    return prediction, average

word = "there"
occurrence = 0.002
prediction, average = get_scores(word, occurrence)

print(f"Word: {word}")
print("Predicted average score via random forests: \t{:0.2f}".format(prediction))
# Print average score according to tweet data if the word exists in it
print(("No data found for this word in tweet data." if average == None else "Average score via tweet data: \t\t\t{:0.2f}".format(average)))

Word: there
Predicted average score via random forests: 	3.66
No data found for this word in tweet data.
